In [2]:
import pandas as pd
import requests as re
import numpy as np
import matplotlib.pyplot as plt

import asyncio
from aiohttp import ClientSession
from loguru import logger


In [4]:
url_general = 'https://www.fundamentus.com.br/resultado.php'
url_paper = 'https://www.fundamentus.com.br/detalhes.php?papel='          

def get_papers():
    res = re.get(url_general)
    
    df = pd.read_html(res.content)[0]
    
    # Filter the companies without recent activity
    df = df[df['Liq.2meses'] != '000']
    
    df.set_index('Papel', inplace=True)
    
    return df[df['Cotação'] > 0]

async def get_paper_info(paper: str):
    logger.info(paper)
    async with ClientSession() as session:
        resp = await session.request(method='GET', url=url_paper+paper)
#         res = re.get(url_paper + paper)
        html = await resp.text()
        df_list = pd.read_html(html)
#     df_list = pd.read_html(res.text)
    
    cleaner = lambda x: x.replace('?', '')\
                         .replace('.', '')\
                         if isinstance(x, str) else x
    
    # Separate and clean diferent tables found on page
    df0 = df_list[0].applymap(cleaner)
    df1 = df_list[1].applymap(cleaner)
    df2 = df_list[2].applymap(cleaner)
    df3 = df_list[3].applymap(cleaner)
    df4 = df_list[4].applymap(cleaner)

    # Drop some headers
    df2.drop(0, inplace=True)
    df3.drop(0, inplace=True)
    df4.drop([0, 1], inplace=True)

    index = pd.concat([
        df0[0][1:],
        df0[2],
        df1[0].append(df1[2]),
        'oscilations_' + df2[0],
        'indicator_' + df2[2].append(df2[4]),
        df3[0].append(df3[2]),
        'demonstrative12m_' + df4[0],
        'demontrative3m_' + df4[2]], ignore_index=True)

    data = df0[1][1:].append([df0[3],
                              df1[1], df1[3],
                              df2[1], df2[3], df2[5],
                              df3[1], df3[3],
                              df4[1], df4[3]]).values

    return pd.Series(index=index, data=data, name=paper)


In [5]:
def save_data(papers):
    holder = list()
    for paper in papers.index:
        try:
            paper_info = get_paper_info(paper)
        except: 
            
            continue
        holder.append(paper_info)
        
    save = pd.DataFrame(holder)
    
    save.to_csv('./data/data.csv')
    
    return save


In [6]:
papers = get_papers()

In [15]:
async def main():
    r = [get_paper_info(paper) async for paper in papers]
    return r

In [16]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [17]:
import time

In [20]:
time.sleep(0.1)

In [21]:
def load_csv(path='./data/data.csv', index_col='Unnamed: 0'):
    data = pd.read_csv(path, index_col=index_col)
    
    return data

In [23]:
df = load_csv(path='partial_result.csv')

In [26]:
a = df[:50].index
b = df[50:].index

In [44]:
a.difference(b).empty

False

In [46]:
with open('somefile.txt', 'w') as f:
    f.write('ola')

In [47]:
with open('sometinhg.txt') as f:
    pass

FileNotFoundError: [Errno 2] No such file or directory: 'sometinhg.txt'

In [39]:
with open('omgDeuCerto.csv', 'w') as f:
    pd.read_csv(f)

UnsupportedOperation: not readable

In [48]:
raise Exception('You are up to date')

Exception: You are up to date

In [55]:
with open('cp_data.csv', 'w') as cp:
    with open('data.csv') as data:
        cp.write(data)
    

TypeError: write() argument must be str, not _io.TextIOWrapper

In [57]:
data.

ValueError: I/O operation on closed file.

In [61]:

if True print('ola')

SyntaxError: invalid syntax (<ipython-input-61-05a9f504cdac>, line 1)

In [71]:
f'somthing {np.round(15.54989186518)}'

'somthing 16.0'

In [72]:
import os

In [73]:
os.remove('cp_data.csv')

In [77]:
while bool():
    print('ola')
    break

In [3]:
!pwd

/home/murilo/Code/pyProjects/InvestCode/scrapper


In [15]:
df = pd.read_csv('partial_data.csv')

In [17]:
df.to_csv('teste.csv', mode='a')

In [18]:
df = pd.read_csv('teste.csv')

In [21]:
df.dropna(subset=['Unnamed: 0']).to_csv('teste.csv')